# Tweet Sentiment NLP

In [1]:
# check if modules are installed
from subprocess import Popen, PIPE

output = Popen("pip list | awk '{print $1}'", shell = True, stdout=PIPE).stdout.read().split()
packages = [x.decode('utf-8') for x in output][2:]
modules = ['kaggle']
for nm in modules:
    if nm not in packages:
        ! pip install {nm}

In [22]:
# importing modules
import glob
import pandas as pd
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

In [18]:
# downloading dataset
api = KaggleApi()
api.authenticate()
dataset = 'kazanova/sentiment140'
csv_name = 'training.1600000.processed.noemoticon.csv'
api.dataset_download_file(dataset, file_name=csv_name, path='./')
fn = glob.glob('train*.zip', recursive = True)[0]
with zipfile.ZipFile(fn) as zip:
    for file in zip.namelist():
        if file == csv_name:
            zip.extract(csv_name)
os.remove(fn)

In [27]:
# importing data
columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df_data = pd.read_csv(csv_name, header = None, names = columns)
print(f'Dataframe dimensions: {df_data.shape}')
df_data.head()

Dataframe dimensions: (1600000, 6)


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
